***GENERATED CODE FOR usstrokemortality PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'}], 'feature': 'LocationAbbr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'TX', 'missing': '0', 'distinct': '26'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationAbbr'}, {'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationAbbr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'}], 'feature': 'LocationDesc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ada County', 'max': 'Yavapai County', 'missing': '0', 'distinct': '314'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationDesc'}, {'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationDesc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GeographicLevel', 'transformation_label': 'String Indexer'}], 'feature': 'GeographicLevel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'County', 'max': 'State', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GeographicLevel'}, {'feature_label': 'GeographicLevel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GeographicLevel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataSource', 'transformation_label': 'String Indexer'}], 'feature': 'DataSource', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'NVSS', 'max': 'NVSS', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataSource'}, {'feature_label': 'DataSource', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataSource')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Class', 'transformation_label': 'String Indexer'}], 'feature': 'Class', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiovascular Diseases', 'max': 'Cardiovascular Diseases', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Class'}, {'feature_label': 'Class', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Class')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Topic', 'transformation_label': 'String Indexer'}], 'feature': 'Topic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Stroke Mortality', 'max': 'Stroke Mortality', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Topic'}, {'feature_label': 'Topic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Topic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Unit', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Unit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'per 100,000 population', 'max': 'per 100,000 population', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Unit'}, {'feature_label': 'Data_Value_Unit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Unit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Age-adjusted, 3-year Average Rate', 'max': 'Age-adjusted, Spatially Smoothed, 3-year Average Rate', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Type'}, {'feature_label': 'Data_Value_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Footnote_Symbol', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Footnote_Symbol', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '231', 'mean': '', 'stddev': '', 'min': '~', 'max': '~', 'missing': '269', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Footnote_Symbo...'}, {'feature_label': 'Data_Value_Footnote_Symbol', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Data_Value_Footnote_Symbol')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Footnote', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Footnote', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '247', 'mean': '', 'stddev': '', 'min': 'Insufficient Data', 'max': 'Insufficient Data', 'missing': '269', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Footnote'}, {'feature_label': 'Data_Value_Footnote', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Footnote')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Gender', 'max': 'Gender', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory1'}, {'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Overall', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification1'}, {'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory2', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Race/Ethnicity', 'max': 'Race/Ethnicity', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory2'}, {'feature_label': 'StratificationCategory2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification2', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'American Indian and Alaskan Native', 'max': 'White', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification2'}, {'feature_label': 'Stratification2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopicID', 'transformation_label': 'String Indexer'}], 'feature': 'TopicID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'T6', 'max': 'T6', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopicID'}, {'feature_label': 'TopicID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopicID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Georeferenced Column', 'transformation_label': 'String Indexer'}], 'feature': 'Georeferenced Column', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'POINT (-100.236 44.4332)', 'max': 'POINT (144.777 13.4431)', 'missing': '0', 'distinct': '398'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Georeferenced Column'}, {'feature_label': 'Georeferenced Column', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Georeferenced Column')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run usstrokemortalityHooks.ipynb
try:
	#sourcePreExecutionHook()

	usstrokemortalityinadultover = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/US Stroke Mortality in adult over 35.csv', 'filename': 'US Stroke Mortality in adult over 35.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(usstrokemortalityinadultover)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run usstrokemortalityHooks.ipynb
try:
	#transformationPreExecutionHook()

	usstrokemortalityautofe = TransformationMain.run(usstrokemortalityinadultover,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "index", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "249.5", "stddev": "144.48", "min": "0", "max": "499", "missing": "0"}, "updatedLabel": "index"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2016.0", "stddev": "0.0", "min": "2016", "max": "2016", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "LocationAbbr", "transformation_label": "String Indexer"}], "feature": "LocationAbbr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "TX", "missing": "0", "distinct": "26"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationAbbr"}, {"transformationsData": [{"feature_label": "LocationDesc", "transformation_label": "String Indexer"}], "feature": "LocationDesc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ada County", "max": "Yavapai County", "missing": "0", "distinct": "314"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationDesc"}, {"transformationsData": [{"feature_label": "GeographicLevel", "transformation_label": "String Indexer"}], "feature": "GeographicLevel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "County", "max": "State", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GeographicLevel"}, {"transformationsData": [{"feature_label": "DataSource", "transformation_label": "String Indexer"}], "feature": "DataSource", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "NVSS", "max": "NVSS", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataSource"}, {"transformationsData": [{"feature_label": "Class", "transformation_label": "String Indexer"}], "feature": "Class", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cardiovascular Diseases", "max": "Cardiovascular Diseases", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Class"}, {"transformationsData": [{"feature_label": "Topic", "transformation_label": "String Indexer"}], "feature": "Topic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Stroke Mortality", "max": "Stroke Mortality", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Topic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Data_Value", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "269", "mean": "81.7", "stddev": "26.38", "min": "2.0", "max": "252.1", "missing": "231"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Data_Value"}, {"transformationsData": [{"feature_label": "Data_Value_Unit", "transformation_label": "String Indexer"}], "feature": "Data_Value_Unit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "per 100,000 population", "max": "per 100,000 population", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Unit"}, {"transformationsData": [{"feature_label": "Data_Value_Type", "transformation_label": "String Indexer"}], "feature": "Data_Value_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Age-adjusted, 3-year Average Rate", "max": "Age-adjusted, Spatially Smoothed, 3-year Average Rate", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Type"}, {"transformationsData": [{"feature_label": "Data_Value_Footnote_Symbol", "transformation_label": "String Indexer"}], "feature": "Data_Value_Footnote_Symbol", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "231", "mean": "", "stddev": "", "min": "~", "max": "~", "missing": "269", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Footnote_Symbo..."}, {"transformationsData": [{"feature_label": "Data_Value_Footnote", "transformation_label": "String Indexer"}], "feature": "Data_Value_Footnote", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "247", "mean": "", "stddev": "", "min": "Insufficient Data", "max": "Insufficient Data", "missing": "269", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Footnote"}, {"transformationsData": [{"feature_label": "StratificationCategory1", "transformation_label": "String Indexer"}], "feature": "StratificationCategory1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Gender", "max": "Gender", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory1"}, {"transformationsData": [{"feature_label": "Stratification1", "transformation_label": "String Indexer"}], "feature": "Stratification1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Overall", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification1"}, {"transformationsData": [{"feature_label": "StratificationCategory2", "transformation_label": "String Indexer"}], "feature": "StratificationCategory2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Race/Ethnicity", "max": "Race/Ethnicity", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory2"}, {"transformationsData": [{"feature_label": "Stratification2", "transformation_label": "String Indexer"}], "feature": "Stratification2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "American Indian and Alaskan Native", "max": "White", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification2"}, {"transformationsData": [{"feature_label": "TopicID", "transformation_label": "String Indexer"}], "feature": "TopicID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "T6", "max": "T6", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopicID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12000.34", "stddev": "6673.76", "min": "4", "max": "48497", "missing": "0"}, "updatedLabel": "LocationID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Y_lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "37.69", "stddev": "7.16", "min": "-14.30175426", "max": "64.90397704", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Y_lat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "X_lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-94.85", "stddev": "19.66", "min": "-170.367", "max": "144.777", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "X_lon"}, {"transformationsData": [{"feature_label": "Georeferenced Column", "transformation_label": "String Indexer"}], "feature": "Georeferenced Column", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "POINT (-100.236 44.4332)", "max": "POINT (144.777 13.4431)", "missing": "0", "distinct": "398"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Georeferenced Column"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "States", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "496", "mean": "24.2", "stddev": "11.7", "min": "4", "max": "51", "missing": "4"}, "updatedLabel": "States"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Counties", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "497", "mean": "1040.97", "stddev": "679.43", "min": "2", "max": "3136", "missing": "3"}, "updatedLabel": "Counties"}]}))

	#transformationPostExecutionHook(usstrokemortalityautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run usstrokemortalityHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(usstrokemortalityautofe, ["index", "Year", "LocationID", "Y_lat", "X_lon", "States", "Counties", "LocationAbbr_stringindexer", "LocationDesc_stringindexer", "GeographicLevel_stringindexer", "DataSource_stringindexer", "Class_stringindexer", "Topic_stringindexer", "Data_Value_Unit_stringindexer", "Data_Value_Type_stringindexer", "Data_Value_Footnote_Symbol_stringindexer", "Data_Value_Footnote_stringindexer", "StratificationCategory1_stringindexer", "Stratification1_stringindexer", "StratificationCategory2_stringindexer", "Stratification2_stringindexer", "TopicID_stringindexer", "Georeferenced Column_stringindexer"], "Data_Value")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

